In [0]:
CREATE TABLE IF NOT EXISTS gold.dim_orders (
  order_sk BIGINT GENERATED ALWAYS AS IDENTITY,
  order_id STRING,
  customer_id STRING,
  order_status STRING,
  order_purchase_timestamp TIMESTAMP,
  order_approved_at TIMESTAMP,
  order_delivered_carrier_date TIMESTAMP,
  order_delivered_customer_date TIMESTAMP,
  order_estimated_delivery_date TIMESTAMP
) 
USING DELTA;

In [0]:
%python
orders_df = spark.read.table("silver.orders")

In [0]:
%python
orders_df.createOrReplaceTempView("updates_view")

In [0]:
MERGE INTO gold.dim_orders s
USING updates_view 
ON s.order_id = updates_view.order_id
WHEN MATCHED
 THEN UPDATE SET 
  s.order_status = updates_view.order_status,
  s.order_purchase_timestamp = updates_view.order_purchase_timestamp,
  s.order_approved_at = updates_view.order_approved_at,
  s.order_delivered_carrier_date = updates_view.order_delivered_carrier_date,
  s.order_delivered_customer_date = updates_view.order_delivered_customer_date,
  s.order_estimated_delivery_date = updates_view.order_estimated_delivery_date
WHEN NOT MATCHED
 THEN INSERT (
  order_id,
  customer_id,
  order_status,
  order_purchase_timestamp,
  order_approved_at,
  order_delivered_carrier_date,
  order_delivered_customer_date,
  order_estimated_delivery_date
 ) VALUES (
  updates_view.order_id,
  updates_view.customer_id,
  updates_view.order_status,
  updates_view.order_purchase_timestamp,
  updates_view.order_approved_at,
  updates_view.order_delivered_carrier_date,
  updates_view.order_delivered_customer_date,
  updates_view.order_estimated_delivery_date
 )

In [0]:
SELECT * FROM gold.dim_orders LIMIT 10;